In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
files = pd.read_csv('result/rsrp-lrt.csv/0.part')
cell_info = pd.read_excel('sites/LTE Cell Config XL-EID 10 May 2023.xlsb', header=2, sheet_name='Cell')
polygon = pd.read_csv('csv_polygon/LRT_Polygon.csv')
polygon_gpd   = gpd.GeoDataFrame(polygon, geometry=gpd.GeoSeries.from_wkt(polygon['WKT']), crs=4326)

In [ ]:

split = files['combined'].str.split(pat='@', expand=True)
split.rename(columns={0:"Site ID", 1:"Polygon_Name", 2:"eNodeB ID", 3:"CI", 4:"Hour", 5:"Shape"}, inplace=True)
files = pd.concat([files, split], axis=1)
files.drop(labels= 'combined', axis=1, inplace=True)
files['Hour'] = files['Hour'].astype(float) + 7 

In [ ]:
files['concatenate'] = files['eNodeB ID'].astype(str).str.replace(pat='.0', repl='') + "|" + files['CI'].astype(str).str.replace(pat='.0', repl='')

files = files.merge(cell_info[['Longitude', 'Latitude']], how='inner', left_on=files['concatenate'], right_on=cell_info['Combine'])

In [ ]:
files_gpd   = gpd.GeoDataFrame(files, geometry=gpd.points_from_xy(files['Longitude'], files['Latitude']), crs=4326)

files_gpd = files_gpd.sjoin_nearest(polygon_gpd, how='inner', max_distance = 0.0008985 * 3)

files_gpd.drop(columns=['WKT', 'id', 'index_right', 'geometry', 'concatenate', 'key_0'], inplace=True)

In [ ]:
files_gpd.to_excel('result/parsing_result_from_dask.xlsx', index=False)